In [1]:
!pip install evaluate
!pip install jiwer
!pip install pymorphy2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 80.7 MB/s eta 0:00:00:00:010:01
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=4cd0f0a806329160f372a226c8c059feed45f5592929759bc892d931613572a0
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
# ипорт нужных библиотек и модулей

import torch
import evaluate
import re
import pymorphy2

from datasets import load_dataset, Audio, DatasetDict
from transformers import WhisperProcessor
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from transformers import WhisperForConditionalGeneration
from functools import partial
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer

In [3]:
# датасет закрытый, нужно авторизоваться

dataset = DatasetDict()

dataset['train'] = load_dataset(
    "mozilla-foundation/common_voice_13_0", "ru", split="train"
)
dataset['test'] = load_dataset(
    "mozilla-foundation/common_voice_13_0", "ru", split="test"
)

dataset = dataset.select_columns(['audio', 'sentence'])

dataset['train'] = dataset['train'].shuffle(seed=42).select(range(20000))
dataset['test'] = dataset['test'].shuffle(seed=42).select(range(4000))

README.md:   0%|          | 0.00/14.7k [00:00<?, ?B/s]

common_voice_13_0.py:   0%|          | 0.00/8.18k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.65k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

The repository for mozilla-foundation/common_voice_13_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_13_0.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


...


Reading metadata...: 10186it [00:00, 126828.08it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 13491it [00:00, 122475.68it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 9523it [00:00, 145181.05it/s]


In [4]:
# предварительная обработка данных

# загрузка процессора
processor = WhisperProcessor.from_pretrained('openai/whisper-small', language='ru', 
                                            task='transcribe')

# передискретизация
sampling_rate = processor.feature_extractor.sampling_rate
dataset = dataset.cast_column('audio', Audio(sampling_rate=sampling_rate))

# подготовка данных перед подачей в модель
def prepare_dataset(ex):
     audio = ex['audio']
     ex = processor(audio=audio['array'], sampling_rate=audio['sampling_rate'],
                    text = ex['sentence'])
     ex['input_length'] = len(audio['array'])/audio['sampling_rate']
     return ex

dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names['train'], num_proc=2)

# т.к. модель принимает аудио=30с, то уберем те, которые короче

max_input_length = 30.0


def is_audio_in_length_range(length):
    return length < max_input_length

dataset["train"] = dataset["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
)

dataset

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Map (num_proc=2):   0%|          | 0/20000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/4000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels', 'input_length'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['input_features', 'labels', 'input_length'],
        num_rows: 4000
    })
})

In [5]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Обработка входных данных
        input_features = [
            {'input_features': feature['input_features'][0]} for feature in features
        ]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors='pt')

        # Обработка меток (labels)
        label_features = [{'input_ids': feature['labels']} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors='pt')  # Явно вызываем tokenizer.pad
        labels = labels_batch['input_ids'].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # Удаление BOS-токена, если он присутствует
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

# Инициализация data_collator
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


In [6]:
# Инициализация нормализатора для русского языка
morph = pymorphy2.MorphAnalyzer()

def normalize_text(text):
    """Нормализация текста: приведение к нижнему регистру, удаление пунктуации, лемматизация"""
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)  # Убираем знаки препинания
    words = text.split()
    normalized_words = [morph.parse(word)[0].normal_form for word in words]
    return " ".join(normalized_words)

# Загрузка WER-метрики
metric = evaluate.load("wer")

def compute_metric(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Заменяем -100 на pad_token_id, чтобы не учитывать паддинг в WER
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # Декодирование предсказаний и истинных меток
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # Оценка обычного (орфографического) WER
    wer_ortho = 100 * metric.compute(predictions=pred_str, references=label_str)

    # Нормализация текста (для русского языка)
    pred_str_norm = [normalize_text(pred) for pred in pred_str]
    label_str_norm = [normalize_text(label) for label in label_str]

    # Убираем пустые строки (чтобы не исказить метрику)
    pred_str_norm = [p for i, p in enumerate(pred_str_norm) if len(label_str_norm[i]) > 0]
    label_str_norm = [l for l in label_str_norm if len(l) > 0]

    # Вычисление нормализованного WER
    wer = 100 * metric.compute(predictions=pred_str_norm, references=label_str_norm)

    return {"wer_ortho": wer_ortho, "wer": wer}


In [7]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

model.config.use_cache = False

model.generate = partial(
    model.generate, language="ru", task="transcribe", use_cache=True
)

# Проверяем устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Убедитесь, что модель на GPU
model = model.to(device)


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

Using device: cuda


In [8]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-ru", 
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=2,
    max_steps=10,
    gradient_checkpointing=False,
    fp16=True,
    fp16_full_eval=False,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=10,
    eval_steps=10,
    logging_steps=5,
    report_to=['tensorboard'],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,  # Включаем пуш на Hugging Face
    hub_model_id="EvgeniaKozhema/whisper-small-ru",  # Укажи свой Hugging Face ID
    hub_strategy="every_save",  # Загружает модель после каждого сохранения
    save_total_limit=3,
)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metric,
    processing_class=processor,
)

trainer.train()
trainer.push_to_hub()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Wer Ortho,Wer
10,2.446300,1.652319,24.390750,13.508493


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.proc

CommitInfo(commit_url='https://huggingface.co/EvgeniaKozhema/whisper-small-ru/commit/280981438e924663069bcb2157f6cdbbc1f579ea', commit_message='End of training', commit_description='', oid='280981438e924663069bcb2157f6cdbbc1f579ea', pr_url=None, repo_url=RepoUrl('https://huggingface.co/EvgeniaKozhema/whisper-small-ru', endpoint='https://huggingface.co', repo_type='model', repo_id='EvgeniaKozhema/whisper-small-ru'), pr_revision=None, pr_num=None)